In [47]:
import spacy
import openai
import json
import os
from dotenv import load_dotenv
from tenacity import retry, stop_after_attempt, wait_random_exponential
from random import sample, seed

In [4]:
nlp = spacy.load("en_core_web_sm")

In [6]:
custom_ner = nlp.add_pipe("ner", name="custom_ner")
custom_ner.add_label("TECHNOLOGY")

1

In [19]:
load_dotenv()

True

## Before putting anything in to the openai api
Set up a binary classifier to identify if a given paragraph has any relevance to the task.  I.e. if it contains any technologies/skills that I might want to take out.  
### First step is to grab a bunch of paragraphs from job descriptions, label a few hundred as relevant or not.

In [29]:
job_desc_list = []
f = open(fr"data/07-09-23-q-data+science-data+analysis-data+engineer-mle-machine+learning-mlops-l-remote.json")
data = json.load(f)
for key in list(data.keys()):
    job_desc_list.append(data[key]['desc'])

In [36]:
len(job_desc_list)/2

260.0

In [65]:
#520 jobs in total here, randomly sample half of them to label. set seed for reproducability
seed(10) 
jobs_to_label = sample(job_desc_list, int(len(job_desc_list)/2))

In [71]:
jobs_to_label[3]

'Remote \n \n \n Rhino is looking for team members who believe that renters deserve greater financial freedom when moving into a home, and that property owners shouldn’t have to stick to the status quo. With over $45 billion tied up in security deposits for 110 million renters in the United States alone, we built nationally recognized technology-driven products to help bring renting into the 21st century. Our solutions help property owners manage their deposits end-to-end, and renters can choose between security deposit insurance or a traditional cash deposit, in addition to renter’s insurance. \n  Rhino’s remote-first culture is driven by our five core values: own your role, raise the bar, be customer obsessed, just be kind, and commit to the Crash (a Crash is a group of Rhinos!). These values help guide us to do our best work and attract talent that helps us achieve our key goals. Our ideal candidate is someone who has worked in a fast-paced, ever-evolving tech environment, and isn’t

In [67]:
[(x.strip()," ") for x in jobs_to_label[2].splitlines() if x.strip() != ""]

[('A career in our Finance team, within our Oracle consulting practice, will provide you with the opportunity to help organizations use enterprise technology to achieve their digital technology goals and capitalise on business opportunities. We help our clients implement and effectively use Oracle offerings to solve their business problems and fuel success in the areas of finance, operations, human capital, customer, and governance, risk and compliance. As part of our finance team, you’ll focus on providing the support companies need in their Finance Transformation journey enabled by Oracle Cloud ERP and EPM. You will be part of a team that helps clients rethink their Finance functions as they leverage new Cloud technology including RPA, Machine Learning and Analytics in conjunction with their ERP platform. You will bring a blend of process and technology expertise to create the next generation Finance function.',
  ' '),
 ('To really stand out and make us fit for the future in a const

### Second step is to featurize these labeled data

In [20]:
api_key = os.getenv("openai_api_key")
example_text_1 = os.getenv("example_text_1")
example_text_2 = os.getenv("example_text_2")

In [12]:
# Originally from getTechs.py
def get_techs(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role":"system", "content":"You identify specific technology names from job descriptions.  Respond only with a list of the names of the specific technologies."},
            {"role":"user", "content":f"Identify the specific technologies in the following text and respond with them in a list: {example_text_1}"},
            {"role":"assistant", "content":"llama 2, python, pytorch"},
            {"role":"user", "content":f"Identify the specific technologies in the following text: {example_text_2}"},
            {"role":"assistant", "content":"python, spark, sql, snowflake, tableau, aws, azure, power bi"},
            {"role":"user", "content":f"Identify the specific technologies in the following text: {text}"}
        ]
    )
    return response

In [9]:
def dict_to_json(dict, filepath):
    with open(filepath, "w") as out:
        json.dump(dict, out)

In [ ]:
# Originally from getTechs.py
# Modified to remove counter, use to just get my spacy training data.

### Does the original json file need to be closed?  I'm unsure how it works when renaming then dumping.  The new file is definitely closed as it's in the with open() command
### Original file may not be? Think about and fix if necessary
def print_attempt_number(retry_state):
    print(f"Retrying: {retry_state.attempt_number}...")
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6), after=print_attempt_number)
def get_tech_list(filepath):
    f = open(filepath)
    data = json.load(f)
    for key in list(data.keys()):
        job_desc = data[key]['desc']
        response = get_techs(job_desc) # keep whole object in case I want metadata from it later
        response_list = [x.lower() for x in response["choices"][0]["message"]["content"].split(", ")]
        data[key]['techs'] = response_list 
    # dict_to_json originally from jobSearch.py, use to indicate which files have already been parsed
    # Right now saves file separately, keeping original.  When done with testing will want it to overwrite.  Then it won't matter if the original is saved or not as we'll delete it anyways.
    dict_to_json(data, f"p-{filepath}")  

In [ ]:
# for filename in os.listdir("data"):
#     if filename.startswith("p-"): #already been parsed
#         continue
#     else:
#         filepath = fr"data/{filename}"
#         get_tech_list(filename) # remember this saves the files separately, keeping the original for now.
        

Above will put the openai decided techs into a list in data[job_id]['techs'].  Can now use data itself for training spacy

In [28]:
# Test with only the first json now
# Hit error with rate limits on openai api
# https://platform.openai.com/docs/guides/rate-limits/error-mitigation
get_tech_list(fr"data/07-09-23-q-data+science-data+analysis-data+engineer-mle-machine+learning-mlops-l-remote.json")

RetryError: RetryError[<Future at 0x1b20bea8af0 state=finished raised RateLimitError>]